In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pykrige.ok import OrdinaryKriging

print("--- Proses Evaluasi Model Dimulai ---")

print("\nLangkah 1: Memuat data dari direktori 'data/'...")
try:
    path_prefix = 'data/'
    df_crime = pd.read_csv(path_prefix + 'kejadian.csv')
    gdf_all_kecamatan = gpd.read_file(path_prefix + 'peta_purbalingga/gadm41_IDN_3.shp')
    print("✓ Data kejadian dan peta kecamatan berhasil dimuat.")
except Exception as e:
    print(f"❌ ERROR: Gagal memuat file. Pastikan file 'kejadian.csv' dan 'peta_purbalingga/gadm41_IDN_3.shp' ada di dalam folder 'data/'.")
    print(f"Detail Error: {e}")

if 'df_crime' in locals() and 'gdf_all_kecamatan' in locals():

    print("\nLangkah 2: Memfilter peta untuk wilayah Purbalingga...")
    kabupaten_target = "Purbalingga"
    gdf_kec_pbg = gdf_all_kecamatan[gdf_all_kecamatan['NAME_2'] == kabupaten_target]

    if gdf_kec_pbg.empty:
        print(f"❌ ERROR: Tidak ditemukan data untuk kabupaten '{kabupaten_target}' dalam shapefile.")
    else:
        print(f"✓ Peta berhasil difilter untuk {kabupaten_target}.")

    print("\nLangkah 3: Mempersiapkan data untuk model...")
    lons = df_crime['longitude'].values
    lats = df_crime['latitude'].values
    values = df_crime['jumlah_kejadian'].values
    print("✓ Data longitude, latitude, dan jumlah kejadian siap digunakan.")

    print("\nLangkah 4: Mencari parameter variogram optimal (sill, range, nugget)...")
    OK_final = OrdinaryKriging(
        lons, lats, values, 
        variogram_model="spherical", 
        verbose=True, 
        enable_plotting=False
    )

    FIXED_PARAMS = list(OK_final.variogram_model_parameters)
    
    print("\n✓ Parameter Variogram (sill, range, nugget) ditemukan:")
    if FIXED_PARAMS and len(FIXED_PARAMS) == 3:
        print(f"  Sill   = {FIXED_PARAMS[0]}")
        print(f"  Range  = {FIXED_PARAMS[1]}")
        print(f"  Nugget = {FIXED_PARAMS[2]}")
    else:
        print("❌ ERROR: Gagal menghitung parameter variogram di Langkah 4. Menghentikan skrip.")
        exit()

    print("\nLangkah 5: Mengevaluasi kinerja model (Leave-One-Out Cross-Validation)...")
    predicted_values = []

    for i in range(len(values)):
        train_lons = np.delete(lons, i)
        train_lats = np.delete(lats, i)
        train_values = np.delete(values, i)
        
        val_lon, val_lat = lons[i], lats[i]

        OK_val = OrdinaryKriging(
            train_lons,
            train_lats,
            train_values,
            variogram_model="spherical",
            variogram_parameters=FIXED_PARAMS, 
            verbose=False, 
            enable_plotting=False
        )

        try:
            val_pred, _ = OK_val.execute("points", val_lon, val_lat)
            predicted_values.append(val_pred[0])
        except Exception as e:
            print(f"Warning: Gagal memprediksi titik ke-{i}. Error: {e}")
            predicted_values.append(np.nan)

    actual = np.array(values)
    predicted = np.array(predicted_values)
    
    valid_indices = ~np.isnan(predicted)
    
    if np.sum(valid_indices) == 0:
        print("❌ ERROR: Semua prediksi gagal. Tidak dapat menghitung MSE/MAPE.")
    else:
        mse = np.mean((actual[valid_indices] - predicted[valid_indices]) ** 2)
        
        epsilon = 1e-10 
        mape = np.mean(np.abs((actual[valid_indices] - predicted[valid_indices]) / (actual[valid_indices] + epsilon))) * 100

        print("\n✓ Evaluasi Selesai. Hasil:")
        print(f" - Mean Squared Error (MSE) = {mse:.4f}")
        print(f" - Mean Absolute Percentage Error (MAPE) = {mape:.2f}%")

    print("\n--- Proses Evaluasi Selesai ---")

else:
    print("\n--- Proses Evaluasi Dihentikan Karena Gagal Memuat Data Awal ---")

--- Proses Evaluasi Model Dimulai ---

Langkah 1: Memuat data dari direktori 'data/'...
✓ Data kejadian dan peta kecamatan berhasil dimuat.

Langkah 2: Memfilter peta untuk wilayah Purbalingga...
✓ Peta berhasil difilter untuk Purbalingga.

Langkah 3: Mempersiapkan data untuk model...
✓ Data longitude, latitude, dan jumlah kejadian siap digunakan.

Langkah 4: Mencari parameter variogram optimal (sill, range, nugget)...
Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'spherical' Variogram Model
Partial Sill: 0.019778211086083672
Full Sill: 0.019778211127509684
Range: 0.016273841497884106
Nugget: 4.142601207191094e-11 

Calculating statistics on variogram model fit...

✓ Parameter Variogram (sill, range, nugget) ditemukan:
  Sill   = 0.019778211086083672
  Range  = 0.016273841497884106
  Nugget = 4.142601207191094e-11

Langkah 5: Mengevaluasi kinerja model (Leave-One-Out Cross-Validation)...

✓ Evaluasi Selesai. Hasil:
 - Mean Square